# Análise dos dados

A análise dos dados de sorteios de um consórcio de imóvel é essencial para entender o comportamento e as tendências dentro do grupo de consorciados. Essa análise permite identificar padrões de contemplação, prever os momentos mais propícios para o sorteio de determinados participantes e melhorar a gestão do consórcio como um todo. Além disso, auxilia na transparência e na tomada de decisões estratégicas, como ajustes nas regras do consórcio, proporcionando mais confiança e previsibilidade para os consorciados. Ao utilizar dados históricos e tendências, é possível aumentar a eficiência e a satisfação dos participantes.



## Os Dados

In [83]:
import pandas as pd
import plotly.express as px

In [84]:
id_cota = 2450765
# [2450765, 2450766, 2450770]

In [85]:
df_contemplados = pd.read_parquet(f"./data/contemplados-{id_cota}.parquet")
df_opcoes_contemplacao = pd.read_parquet(f"./data/opcoes_contemplacao-{id_cota}.parquet")

### Agrupando dados relevantes

Aqui vamos fazer um dataframe onde a data da assembleia, cota, versao e quantidade de parcelas são únicos
entre contemplados e não contemplados. Precisamos juntar as informações e evitar possíveis duplicidades ou 
incoerências, onde por exemplo, um contemplado não faz parte das opções de contemplação.

Alem disso, vamos adicionar:

- **max_parcelas**: O número máximo possível de um lance que é o número de parcelas remanescentes do consórcio
- **max_viavel**: O número de parcelas de lance que seria equivalente a um financiamento imobilário.

In [86]:
df_contemplados["contemplado"] = 1
df_opcoes_contemplacao["contemplado"] = 0

colunas_relevantes = [
    "assembleia",
    "data_assembleia",
    "cota",
    "versao",
    "quantidadeParcelaLance",
    "modalidade",
    "contemplado",
]

df = pd.concat(
    [
        df_contemplados[colunas_relevantes],
        df_opcoes_contemplacao[colunas_relevantes],
    ]
)
df = df.groupby(colunas_relevantes[:-2]).max().reset_index()
df["max_parcelas"] = df["assembleia"].apply(lambda x: 216 - x)
df["max_selic"] = df["assembleia"].apply(lambda x: 152 - x)
df["max_fin_imob"] = df["assembleia"].apply(lambda x: 157 - x)
df["max_sem_perda"] = df["assembleia"].apply(lambda x: 175 - x)
df["str_cota"] = df.apply(lambda x: f"{x['cota']}-{x['versao']}", axis=1)
# df

In [87]:
df["cota"].max()

np.int64(650)

## Evolução das Cotas

Aqui utilizamos as opções de contamplação com apenas uma modalidade por cota.

### Opções de Contemplação

In [88]:
# uma seq. contemplacao por cota
df_chart = (
    df.groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index()
    .sort_values("str_cota", ascending=False)
)
fig = px.bar(
    df_chart,
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
    title=f"Opções de Contemplação por data de assembleia",
)
fig.show()

> **Entender**: São 650 cotas no grupo. Como o número de opções de contemplação em 1200 é táo maior?

### Cotas Ativas

In [89]:
# uma seq. contemplacao por cota
df_chart = (
    df.where(df["modalidade"] != "Sort. Canceladas")
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index()
    .sort_values("str_cota", ascending=False)
)
fig = px.bar(
    df_chart,
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
    title=f"Opções de Contemplação (Sem Canceladas) por data de assembleia",
)
fig.show()

In [90]:
# uma seq. contemplacao por cota
df_chart = df[['data_assembleia', 'modalidade', 'str_cota']]
df_chart["bin_modalidade"] = df_chart["modalidade"].apply(lambda x: "Cancelada" if x == "Sort. Canceladas" else "Valida")
df_chart = (
    df_chart
    .groupby(["data_assembleia", "bin_modalidade"])
    .count()
    .reset_index()
    .sort_values("str_cota", ascending=False)
)
fig = px.bar(
    df_chart,
    x="data_assembleia",
    y="str_cota",
    color="bin_modalidade",
    labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
    title=f"Válidas e Canceladas por data de assembleia",
)
fig.show()

/tmp/ipykernel_1146/2297073132.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



> **Entender**: Não existe uma linha decrescente nas opções de contemplação como seria esperado de um consórcio

## Contemplações

In [91]:
fig = px.bar(
    df[["data_assembleia", "modalidade", "str_cota"]]
    .where(df["contemplado"] == 1)
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index(),
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    title=f"Contemplados por data de assembleia",
    labels={"str_cota": "Quantidade de cotas", "data_assembleia": "Data da assembleia"},
)
fig.show()

### Total de Contemplados

In [92]:
total_contemplados = {}


def cumsum_modalidade(x):
    data_assembleia = x["data_assembleia"]
    modalidade = x["modalidade"]
    contemplados = x["contemplado"]
    total_data_assembleia = total_contemplados.get(data_assembleia) or {}
    total_modalidade = total_data_assembleia.get(modalidade) or 0
    total_modalidade += contemplados
    total_data_assembleia[modalidade] = total_modalidade
    total_contemplados[data_assembleia] = total_data_assembleia


df.apply(cumsum_modalidade, axis=1)
df_total_comtemplados= pd.DataFrame(total_contemplados)
df_total_comtemplados

,2020-10-15T00:00:00,2020-11-12T00:00:00,2020-12-14T00:00:00,2021-01-14T00:00:00,2021-02-12T00:00:00,2021-03-12T00:00:00,2021-04-14T00:00:00,2021-05-13T00:00:00,2021-06-14T00:00:00,2021-07-15T00:00:00,...,2023-11-14T00:00:00,2023-12-14T00:00:00,2024-01-12T00:00:00,2024-02-15T00:00:00,2024-03-14T00:00:00,2024-04-12T00:00:00,2024-05-14T00:00:00,2024-06-12T00:00:00,2024-07-12T00:00:00,2024-08-14T00:00:00
Sorteio Ativas,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Lance Fixo,0,0,1,1,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,1,1
Sort. Canceladas,0,0,0,1,1,0,1,0,0,1,...,8,2,3,4,10,1,0,1,1,2
Lance Livre,1,0,2,2,1,1,1,0,0,0,...,1,1,0,1,1,1,1,1,1,1


In [93]:
fig = px.bar(
    df[["data_assembleia", "modalidade", "str_cota"]]
    .where(df["contemplado"] == 1)
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index(),
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    title=f"Contemplados por data de assembleia",
    labels={"str_cota": "Quantidade de cotas", "data_assembleia": "Data da assembleia"},
)
fig.show()

## Lances Livres

df["max_parcelas"] = df["assembleia"].apply(lambda x: 216 - x)
df["max_selic"] = df["assembleia"].apply(lambda x: 152 - x)
df["max_fin_imob"] = df["assembleia"].apply(lambda x: 157 - x)
df["max_sem_perda"] = df["assembleia"].apply(lambda x: 175 - x)

In [124]:
chart_df = (
    df[["data_assembleia", "max_parcelas", "max_selic", "max_fin_imob", "max_sem_perda", "quantidadeParcelaLance"]]
    .where(df["modalidade"] == "Lance Livre")
    .groupby(["data_assembleia", "max_parcelas", "max_selic", "max_fin_imob", "max_sem_perda"])
    .max()
    .reset_index()
)
# print(chart_df)
fig = px.bar(
    x=chart_df["data_assembleia"],
    y=chart_df["quantidadeParcelaLance"],
    labels={
        "data_assembleia": "Data da assembleia",
        "quantidadeParcelaLance": "Lance Livre",
    },
    title=f"Máximo Lance Livre Ofertado por Data de Assembléia",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_parcelas"],
    mode="lines",
    name="Parcelas Remanescentes",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_sem_perda"],
    mode="lines",
    name="Max sem Retorno Negativo",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_fin_imob"],
    mode="lines",
    name="Taxa eq. Financiamento (13%)",
)

fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_selic"],
    mode="lines",
    name="Taxa eq. SELIC (10,5%)",
)
fig.show()

In [125]:
chart_df = (
    df[["data_assembleia", "max_parcelas", "max_selic", "max_fin_imob", "max_sem_perda", "quantidadeParcelaLance"]]
    .where((df["modalidade"] == "Lance Livre") & (df["contemplado"] == 1))
    .groupby(["data_assembleia", "max_parcelas", "max_selic", "max_fin_imob", "max_sem_perda"])
    .max()
    .reset_index()
)
# print(chart_df)
fig = px.bar(
    x=chart_df["data_assembleia"],
    y=chart_df["quantidadeParcelaLance"],
    labels={
        "data_assembleia": "Data da assembleia",
        "quantidadeParcelaLance": "Lance Livre",
    },
    title=f"Lance Livre Contemplado por Data de Assembléia",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_parcelas"],
    mode="lines",
    name="Parcelas Remanescentes",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_sem_perda"],
    mode="lines",
    name="Max sem Retorno Negativo",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_fin_imob"],
    mode="lines",
    name="Taxa eq. Financiamento (13%)",
)

fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_selic"],
    mode="lines",
    name="Taxa eq. SELIC (10,5%)",
)

fig.show()

## Versão de Cotas

Como visto nos gráficos anteriores, é possível que uma cota tenha mais de uma versão para que as opções 
de conteplação somem mais do que as 650 cotas originais.

In [77]:
modalidades = df["modalidade"].unique()

for modalidade in modalidades:
    df_chart = (
        df.where(df["modalidade"] == modalidade)
        .groupby(["data_assembleia", "versao"])
        .count()
        .reset_index()
        .sort_values("versao")
    )
    fig = px.bar(
        df_chart,
        x="data_assembleia",
        y="str_cota",
        color="versao",
        labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
        title=f"Opções de Contemplação {modalidade} por data de assembleia",
    )
    fig.show()

In [78]:
fig = px.bar(
    df.groupby(["data_assembleia", "versao"]).count().reset_index(),
    x="data_assembleia",
    y="cota",
    color="versao",
    labels={
        "data_assembleia": "Data da assembleia",
        "versao": "Versão",
        "cota": "Número de cotas",
    },
    title=f"Número de cotas por versão",
)
fig.show()

## Número de Contemplações  

In [79]:
df["total_contemplados"] = df["contemplado"].cumsum()

In [80]:
df_linear = df[["data_assembleia"]].drop_duplicates()
df_linear["lin_contemplacoes"] = 650 / 219
df_linear["tot_lin"] = df_linear["lin_contemplacoes"].cumsum()
# df_linear

In [81]:
chart_df = (
    df[["data_assembleia", "total_contemplados"]]
    .groupby("data_assembleia")
    .max()
    .reset_index()
)
fig = px.line(
    chart_df,
    x="data_assembleia",
    y="total_contemplados",
    labels={
        "data_assembleia": "Data da assembleia",
        "total_contemplados": "Número de contemplações",
    },
    title=f"Total de Contemplações",
)
fig.add_hline(
    y=650,
    line_dash="dot",
    annotation_text="Total de cotas",
    annotation_position="bottom right",
)
fig.add_scatter(
    x=df_linear["data_assembleia"],
    y=df_linear["tot_lin"],
    mode="lines",
    name="Se fossem distribuídas linearmente",
)
fig.show()